In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from pathlib import Path
from json import dump, load
from pprint import pprint
from datetime import datetime
from operator import mul
from functools import reduce

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Modules

In [1]:
from subprocess import PIPE, run, STDOUT
from os import environ

#environ['TOMA'] = 'BBBBB'
print(run('module list', stdout=PIPE, stderr=STDOUT, shell=True).stdout.decode('utf-8'))

/bin/sh: module: command not found



# Setup & Run 

## Linear system

In [47]:
params = {
    'Linear system': {
        'Name': 'laplace',
        'Domain partitions': [[0, 120], [0, 120], [0, 120]],
        'Initial guess': 'zero',
        'Exact solution': 'white-noise sine'
    },
    'Solver': { # nested solver
        'Name': '',
        'Steps': '',
        'Max iterations':,
        'Tolerance': '',
        'Timeout [min]':'',
        'Solver': {
            # other parameters
            # ....
        }   
    }
    'Results': {
        
    }
}
params['Device'] = 'julia'
params['Executable path'] = Path('/gpfs/homeb/zam/tnikolov/asynchronator/build/example/')
params['Processes grid'] = [len(x) - 1 for x in params['Domain partitions']]
params['Processes per node'] = 1
params['Threads per process'] = 256
params['Nodes'] = reduce(mul, params['Processes grid'], 1)
params['Timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M")

params

SyntaxError: invalid syntax (<ipython-input-47-6277fbe4e84c>, line 11)

## Solver

### Jacobi

In [43]:
params['Solver'] = 'jacobi'
params['Timeout [min]'] = 4
params['Tolerance'] = 1e-04
params['Steps'] = 10
params

{'Domain partitions': [[0, 120], [0, 120], [0, 120]],
 'Exact solution': 'white-noise sine',
 'Initial guess': 'zero',
 'Operator': 'laplace',
 'Solver name': 'jacobi',
 'Steps': 10,
 'Timeout [min]': 4,
 'Tolerance': 0.0001}

### FCG Jacobi 

In [46]:
params['Solver'] = 'fcg_jacobi'
params['Preconditioner'] = {
    'Steps' : 10,
    'Max iterations' : 10,
    'Timeout [min]' : 4,
    'Tolerance' : 1e-04,
}
params['Steps'] = 10
params['Max iterations'] = 10
params['Timeout [min]'] = 4
params['Tolerance'] = 1e-04

params['Truncation'] = 10
pprint(params)

{'Domain partitions': [[0, 120], [0, 120], [0, 120]],
 'Exact solution': 'white-noise sine',
 'Initial guess': 'zero',
 'Operator': 'laplace',
 'Preconditioner': {'Max iterations': 10,
                    'Steps': 10,
                    'Timeout [min]': 4,
                    'Tolerance': 0.0001},
 'Solver': 'fcg_jacobi',
 'Solver name': 'jacobi',
 'Steps': 10,
 'Timeout [min]': 4,
 'Tolerance': 0.0001,
 'Truncation': 10}


### Block-Jacobi

In [27]:
params['Solver'] = 'blockjacobi'
params['Inner steps'] = 1
params['Inner tolerance'] = 0.1
params['Inner timeout [min]'] = 0.1
params['Outer steps'] = 5
params['Outer tolerance'] = 1e-04
params['Outer timeout [min]'] = 4
params

{'Domain partitions': [[0, 120], [0, 120], [0, 120]],
 'Executable name': 'laplace_blockjacobi',
 'Inner steps': 1,
 'Inner timeout [min]': 0.1,
 'Inner tolerance': 0.1,
 'Outer steps': 5,
 'Outer timeout [min]': 4,
 'Outer tolerance': 0.0001}

### FCG Block-Jacobi

In [ ]:
params['Solver'] = 'fcg_blockjacobi'
params['Preconditioner inner steps'] = 1
params['Preconditioner inner tolerance'] = 
params['Preconditioner inner timeout [min]'] = 
params['Preconditioner outer steps'] = 10
params['Preconditioner timeout [min]'] = 0.1
params['Preconditioner tolerance'] = 0.1
params['Steps']: 10
params['Timeout [min]']: 4
params['Tolerance']: 1e-04
params['Truncation']: 10
params

### Asynchronous Jacobi

In [44]:
params['Solver'] = 'ajacobi'
params['Timeout [min]'] = 4
params['Tolerance'] = 1e-04
params

{'Domain partitions': [[0, 120], [0, 120], [0, 120]],
 'Exact solution': 'white-noise sine',
 'Initial guess': 'zero',
 'Operator': 'laplace',
 'Solver': 'ajacobi',
 'Solver name': 'jacobi',
 'Steps': 10,
 'Timeout [min]': 4,
 'Tolerance': 0.0001}

## parameters JULIA 

In [21]:

params

{'Device': 'JULIA',
 'Domain partitions': [[0, 120], [0, 120], [0, 120]],
 'Executable name': 'laplace_jacobi',
 'Executable path': PosixPath('/gpfs/homeb/zam/tnikolov/asynchronator/build/example'),
 'Nodes': 1,
 'Processes grid': [1, 1, 1],
 'Processes per node': 1,
 'Steps': 20,
 'Threads per process': 256,
 'Timeout [min]': 4,
 'Timestamp': '2018-04-04 14:48:33',
 'Tolerance': 0.0001}

## sbatch job

In [23]:
job = ('sbatch '
       '--job-name={executable} '
       '--nodes={nodes} '
       '--ntasks-per-node={procs} '
       '--cpus-per-task={threads} '
       '--time={timeout} '
       '--mail-user=teodor.nikolov22@gmail.com '
       '--mail-type=FAIL '
       '--error={executable}.err '
       '--output={executable}.out '
       '--wrap=\"srun {execDir}/{executable} \"').format(
    executable=params['Executable name'],
    execDir=params['Executable path'],
    procs=params['Processes per node'],
    nodes=params['Nodes'],
    threads=params['Threads per process'],
    timeout='{:0>2d}:{:0>2d}:00'.format(params['Timeout [min]'] // 60, 
                                        params['Timeout [min]'] % 60)
)
pprint(job)

('sbatch --job-name=laplace_jacobi --nodes=1 --ntasks-per-node=1 '
 '--cpus-per-task=256 --time=00:04:00 --mail-user=teodor.nikolov22@gmail.com '
 '--mail-type=FAIL --error=laplace_jacobi.err --output=laplace_jacobi.out '
 '--wrap="srun '
 '/gpfs/homeb/zam/tnikolov/asynchronator/build/example/laplace_jacobi "')


## submit job

In [16]:
date = datetime.now()
runDir = Path.cwd() / '{}-{}-{}'.format(execPath.name, 
                                        date.strftime("%Hhh%Mmm%Sss"), 
                                        date.strftime("%ddd%mmm%yyy"))
runDir.mkdir(exist_ok=False)
with (runDir / '{}.json'.format(execPath.name)).open('w') as setupF: dump(params, setupF)
run(job, cwd=runDir, shell=True)

('sbatch --job-name=laplace_jacobi --nodes=1 --ntasks-per-node=1 '
 '--cpus-per-task=256 --time=00:04:00 --mail-user=teodor.nikolov22@gmail.com '
 '--mail-type=FAIL --error=laplace_jacobi.err --output=laplace_jacobi.out '
 '--wrap="srun '
 '/gpfs/homeb/zam/tnikolov/asynchronator/build/example/laplace_jacobi "')
{'Device': 'julia',
 'Domain partitions': [[0, 120], [0, 120], [0, 120]],
 'Nodes': 1,
 'Procs': 1,
 'Procs grid': [1, 1, 1],
 'Procs per node': 1,
 'Steps': 20,
 'Threads per proc': 256,
 'Timeout [min]': 4,
 'Timestamp': '2018-02-15 22:01:57',
 'Tolerance': 0.0001}
